## Changing the String Representation of Instances

In [ ]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)
    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)

: 

## Using the format option

In [3]:
_formats = {
'ymd' : '{d.year}-{d.month}-{d.day}',
'mdy' : '{d.month}/{d.day}/{d.year}',
'dmy' : '{d.day}/{d.month}/{d.year}'
}


class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    def __format__(self, code) -> str:
        if code == '':
            code = 'dmy'
        fmt = _formats[code]
        return fmt.format(d=self)

In [4]:
d = Date(2012, 12, 21)
format(d)

'21/12/2012'

In [12]:
from datetime import date

d = date(2012,12,21)
format(d)

'2012-12-21'

In [13]:
format(d,'%A, %B %d, %Y')

'Friday, December 21, 2012'

## Making Objects Support the Context-Management

In [16]:
from socket import socket, AF_INET, SOCK_STREAM


class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
         self.address = address
         self.family = family
         self.type = type
         self.connection = None

    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock

    def __exit__(self):
        self.sock.close()
        self.sock = None

In [ ]:
from functools import partial

conn = LazyConnection(('www.python.org',80))
# Connection closed

with conn as s:
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))


## Saving memory when creating a large number of instances

In [19]:
class Date:
    __slots__ = ['year', 'month', 'day']
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day

## Public and private methods

In [ ]:
class A:
    def __init__(self):
        self._internal = 0
        self.public = 1

    def _internal_method(self):
        '''
        An internal method
        '''

    def public_method(self):


## Encapsulating class names

In [22]:
class A:    #inherits B
    def __init__(self):
        self._internal = 0
        self.public = 1
    
    def public_method(self):
        '''
        #doest override B
        # '''

    def _internal_method(self):
        ...

class B(A):    #inherits B
    def __init__(self):
        self._internal = 0
        self.public = 1
    
    def public_method(self):
        '''
        #doest override B
        # '''

    def _internal_method(self):
        ...


Creating Managed Attributes

In [10]:
# A simple way to customize access to an attribute is to define it as a property

class Person:
    def __init__(self, first_name):
        self.first_name= first_name

    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self,value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

    @first_name.deleter
    def first_name(self):
        raise AttributeError("Cant delete attribute")
        

In [14]:
a = Person('Guido')
a.first_name
a.first_name = 32

TypeError: Expected a string

In [4]:
class Base:
    def __init__(self):
        print("Base.__init__")

class A(Base):
     def __init__(self):
         Base.__init__(self)
         print("A.__init__")

class B(Base):
     def __init__(self):
         Base.__init__(self)
         print("B.__init__")

class C(A, B):
     def __init__(self):
         A.__init__(self)
         B.__init__(self)
         print("C.__init__")

c  = C()

Base.__init__
A.__init__
Base.__init__
B.__init__
C.__init__


## Extending a property in a subclass

In [7]:
class Person:
    def __init__(self, name):
        self.name = name

    # Getter function
    @property
    def name(self):
        return self._name
    @name.setter
    def name(self,value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')

        self._name = value

    @name.deleter
    def name(self):
        raise AttributeError("cant delete attribute")


    

arka = Person("arka")

In [9]:
arka._name = "Arka Prava"

In [12]:
# Here is an example of a class that inherits from Person and extends the name property
# with new functionality
    
class SubPerson(Person):
        @property
        def name(self):
            print('Getting Name')
            return super().name

        @name.setter
        def name(self,value):
            print("setting name")
            super(SubPerson,SubPerson).name.__set__(self,value)

        @name.deleter
        def name(self):
            print('Deleting name')
            super(SubPerson,SubPerson).name.__delete__(self)


In [14]:
s = SubPerson('Arka')
s.name

setting name
Getting Name


'Arka'

## Using lazily computed properties

In [15]:
class lazyproperty:
    def __init__(self,func):
        self.func = func

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value

In [16]:
import math
class Circle:
    def __init__(self, radius):
        self.radius = radius
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2
    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius

In [17]:
c  = Circle(4.0)
c.radius

4.0

## Simplifying the Initialization of Data Structures 

In [7]:
class Structure:
    # Class variable that specifies expected fields
    _fields = []
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments')

        # Set the arguments
        for name, value in zip(self._fields, args):
            setattr(self,name,value)

In [22]:
class Stock(Structure):
    _fields = ['name','shares','price']

s = Stock('ACME',50, 91.1)


TypeError: __init__() takes 1 positional argument but 4 were given

## Same thing used with keywords

In [14]:
class Structure:
    _fields = []
    def __init__(self, *args, **kwargs) -> None:
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))

        for name, value in zip(self._fields, args):
            setattr(self, name, value)

        #Set the additional arguments 
        extra_args = kwargs.keys() - self._fields
        for name in extra_args:
            setattr(self, name, kwargs.pop(name))
        if kwargs:
            raise TypeError('Duplicate values for {}'.format(','.join(kwargs)))

In [18]:
s1 = Stock('ACME',50, 91.1)
s2 = Stock('ACME',50, 91.1, date = '8/2/2012')
s2.date

'8/2/2012'

## Alternate ways to perform these steps

In [21]:
class Structure:
    _fields = []
    def __init__(self) -> None:
        if len(args) != len(self._fields):    
            raise TypeError('Expected {} arguments'.format(len(self._fields)))

        # set the argumetns (alternate)
        self.__dict__.update(zip(self._fields, args))

## Why perform these steps?

In [ ]:
# The above steps allows us to avoid the repetitive 
# self.xxx, self.xxx .... assignments

## for example
class Stock:
def __init__(self, name, shares, price):
    self.name = name
    self.shares = shares
    self.price = price

class Point:
def __init__(self, x, y):
    self.x = x
    self.y = y
    
class Circle:
def __init__(self, radius):
    self.radius = radius



## Implementing a Data Model or Type System

In [15]:
# Base class. Uses a descriptior to set a value
class Descriptor:
    def __init__(self,name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)

    def __set__(self, instance, value):
        instance.__dict__[self.name] = value


# Descriptor for enforcing types
class Typed(Descriptor):
    expected_type = type(None)

    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('expected' + str(self.expected_type))
        super().__set__(instance, value)

# Descriptor for enforcing values
class Unsigned(Descriptor):
    def __set__(self, instance, value):
        if value<0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)


class MaxSized(Descriptor):
    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
            super().__init__(name, **opts)

    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
            super().__set__(instance, value)

In [16]:
class Integer(Typed):
    expected_type = int


class UnsignedInteger(Integer, Unsigned):
    pass

class Float(Typed):
    expected_type = float

class UnsignedFloat(Float, Unsigned):
    pass

class String(Typed):
    expected_type = str

class SizedString(String, MaxSized):
    pass


class Stock:
# Specify constraints
    name = SizedString('name',size=8)
    shares = UnsignedInteger('shares')
    price = UnsignedFloat('price')
    def __init__(self) -> None:
        self.name = name
        self.shares = shares
        self.price = price

In [17]:
s = Stock('ACME', 50, 91.1)

TypeError: __init__() takes 1 positional argument but 4 were given

In [20]:
#other techniques

#class decorator to apply constraints

def check_attributes(**kwargs):
    def decoraddte(cls):
        for key, value in kwargs.items():
            if isinstance(value, Descriptor):
                value.name = key
                setattr(cls, key, value)
            else:
                setattr(cls, key, value(key))
        return cls
    return decoraddte

In [23]:
#Example of the above
@check_attributes(name=SizedString(size=8),
                  shares = UnsignedInteger,
                  price = UnsignedFloat)
class Stock:
    def __init__(self,name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [ ]:
#Another approach to simplify the specification of constraints is to use a metaclass



## Delegating Attribute Access

In [24]:
class A:
    def spam(self):
        pass
    def foo(self):
        pass

class B:
    def __init__(self):
        self._a = A()

    def spam(self, x):
        # Delegate to the internal self._a instance
        return self._a.spam(x)

    def foo(self):
        # Delegate to the internal self._a instance
        return self._a.foo()
    
    def bar(self):
        pass

In [33]:
# Similar but convenient way to delgate to B
class A:
    def spam(self,x):
        print('A.spam')
    def foo(self):
        pass


class B:
    def __init__(self) -> None:
        self._a  = A()

    def spam(self,x):
        print('B.spam')

    def __getattr__(self,name):
        return getattr(self._a, name)
    

# Note that getattr only gets called when the attribute is not found in the class
# This is the case when we try to access the attribute spam
b = B()
b.spam(43)   # INVOKES THE GETATTR FUNCTION TO RETREIVE ALL TEH PROCESSES OF CLASS B

B.spam


## Defining more than one constructor in a class

In [1]:
# You are writing a class, but you want users to be able to create instances in more than the one way provided by __init__()

In [14]:
import time

class Date:
    # Primary const
    def __init__(self, year, month, day):
        self.year =year
        self.month = month
        self.day = day

    # Alternate constructor
    def today():
        t = time.localtime()
        return t.tm_year, t.tm_mon, t.tm_mday

In [15]:
a = Date(2012,12,21)
b = Date.today()

## Create and Instance without invoking init

In [21]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month  = month
        self.day  = day

d = Date.__new__(Date)

In [23]:
d.year = 2019

In [26]:
data = {'year':2012, 'month':12, 'day': 4}
for key, value in data.items():
    setattr(d, key, value)




In [27]:
d.__dict__

{'year': 2012, 'month': 12, 'day': 4}

In [28]:
from time import localtime

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    @classmethod
    def today(cls):
        d = cls.__new__(cls)
        t = localtime()
        d.year = t.tm_year
        d.month = t.tm_mon
        d.day = t.tm_day
        return d

Extending Classes with Mixins

In [1]:
# You have a collection of generally useful methods that you would like tomake available for extending the functionanlity of other class definitions. However the classes where the methods might  be addesd arent necessarily related to one anothe via inheritance. Thus you cant just attach the methods to a common base class.

In [5]:
# Instantiating the following classes does nothing but throw exceptions as they dont have any init


class LoggedMappingMixin:
    ''' 
    Add logging to getsetdelete operations for debugging/.'''

    # Note that it also calls its super class
    # So it expects a superclass
    __slots__ = ()
    def __getitem__(self, key):
        print('Getting ' + str(key))
        return super().__getitem__(key)

    def __setitem__(self, key, value):
        print('Setting {} = {} '.format(str(key), str(value)))
        return super().__setitem__(key, value)

    def __delitem__(self, key):
        print('Deleting {}'.format(str(key)))
        return super().__delitem__(key)


class LoggedDict(LoggedMappingMixin, dict):
    pass

d = LoggedDict()
d['x'] = 23

Setting x = 23 


In [6]:
d['x']

Getting x


23

In [7]:
del d['x']

Deleting x


In [14]:
# set key only once
class SetSingleKey():
    ''' Set key only once '''
    __slots__ = ()
    def __setitem__(self, key, value):
        if key in self:
            raise KeyError(str(key)+' already set')
        return super().__setitem__(key, value)



In [15]:
from collections import defaultdict

class SetOnceDefaultDict(SetSingleKey, defaultdict):
    pass

d = SetOnceDefaultDict(list)

In [16]:

d['x'] = 12


In [17]:
d['x'] = 33

KeyError: 'x already set'

In [19]:
# Alternative way of implementing mixins

def LoggedMapping(cls):
    cls_getitem = cls.__getitem__
    cls_setitem = cls.__setitem__
    cls_delitem = cls.__delitem__
    def __getitem__(self, key):
        print('Getting ' + str(key))
        return cls_getitem(self, key)
    def __setitem__(self, key, value):
        print('Setting {} = {!r}'.format(key, value))
        return cls_setitem(self, key, value)
    def __delitem__(self, key):
        print('Deleting ' + str(key))
        return cls_delitem(self, key)
    cls.__getitem__ = __getitem__
    cls.__setitem__ = __setitem__
    cls.__delitem__ = __delitem__
    return cls

Implementing Stateful Objects or State Machines



In [4]:
class Connection:
    def __init__(self):
        self.state == 'CLOSED'

    def read(self):
        if self.state == 'CLOSED':
            raise RuntimeError('Not Open')
        print('reading')

    def write(self, data):
        if self.state != 'OPEN':
            raise RuntimeError('Not Open')
        print('Writing')

    def open(self):
        if self.state  == 'OPEN':
            raise RuntimeError('Already Open')
        print('Opening')
        self.state = 'OPEN'

    def close(self):
        if self.state == 'CLOSED':
            raise RuntimeError('Already Closed') 
        print('Closed')
        self.state = 'CLOSED'


# Thats too much work, A more elegant way to do this is


class Connection:
    def __init__(self):
        self.new_state(ClosedConnectionState)
        
    def new_state(self, newstate):
        self._state = newstate


    # Delegate to the state class
    def read(self):
        return self._state.read(self)
    
    def write(self, data):
        return self._state.write(self, data)
    def open(self):
        return self._state.open(self)
    def close(self):
        return self._state.close(self)

In [5]:
class ConnectionState:
    @staticmethod
    def read(conn):
        raise NotImplementedError()
    
    @staticmethod
    def write(conn, data):
        raise NotImplementedError()
    
    @staticmethod
    def open(conn):
        raise NotImplementedError()
    
    @staticmethod
    def close(conn):
        raise NotImplementedError()
    
class ClosedConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        raise RuntimeError('Not Open')
    
    @staticmethod
    def write(conn, data):
        raise RuntimeError('Not Open')

    @staticmethod
    def open(conn):
        conn.new_state(OpenConnectionState)

    @staticmethod
    def close(conn):
        raise RuntimeError('Already closed')


class OpenConnectionState(ConnectionState):
    @staticmethod
    def read(self):
        print('reading')
    
    @staticmethod
    def write(conn, data):
        print('writing')

    @staticmethod
    def open(conn):
        raise RuntimeError('Already Open')

    @staticmethod
    def close(conn):
        conn.new_state(ClosedConnectionState)


# The above and the following are not the real world use case implementations of the open and close functions

In [13]:
c = Connection()
c._state = OpenConnectionState

In [14]:
c.read()

reading


In [16]:
class State:
    def __init__(self):
        self.state = 'A'
    def action(self, x):
        if state == 'A':
# Action for A
...
        state = 'B'
        elif state == 'B':
# Action for B
...
            state = 'C'
        elif state == 'C':
# Action for C
...
            state = 'A'

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 13)

In [ ]:
#Alternative implementation

class State:
    def __init__(self):
        self.new_state(State_A)

    def new_state(self, state):
        self.__class__ = state

    def action(self, x):
        raise NotImplementedError()
    
class State_A(State):
    def action(self, x):

        # Some actions for State_A

        
        self.new_state(State_B)

    

## Calling a Method on an Object Given the Name As a String

In [17]:
import math

class Point:
    def __init__(self, x,y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return 'Points {} {}'.format(self.x, self.y)

    def distance(self, x, y):
        return math.hypot(self.x - x, self.y - y)

p = Point(2,3)
d = getattr(p, 'distance')(3,4)
print(d)

1.4142135623730951


In [18]:
import operator
operator.methodcaller('distance', 0, 0)(p)


#Use case

points = [
    Point(1, 2),
    Point(3, 0),
    Point(10, -3),
    Point(-5, -7),
    Point(-1, 8),
    Point(3, 2)
]

points.sort(key=operator.methodcaller('distance', 0, 0))

## Implementing the Visitor Pattern

In [19]:
# You need to write code that processes or navigatest through a complicated data structure consisting of many different kinds of objects each of which needxs to be handled in a different way. For example walking through a tree struture and performing different actions depending on what kind of tree nodew are encountered.

In [22]:
class Node:
    pass

class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right


class Sub(BinaryOperator):
    pass


class Number(Node):
    def __init__(self, value):
        self.value = value

t1 = Sub(1, 2)
print(t1.left)
print(t1.right)

1
2


Implimentations in cyclic stat structtures

In [10]:
import weakref


class Node:
    def __init__(self, value):
        self.value = value
        self._parent = None
        self.children = []

    def __repr__(self):
        return "Node({!r:})".format(self.value)


    @property
    def parent(self):
        return self._parent if self._parent is None  else self._parent()

    @parent.setter
    def parent(self, node):
        self._parent = weakref.ref(node)

    def add_child(self, child):
        self.children.append(child)
        child.parent = self


#implementation
root =  Node('parent/root')
root.children

[]

In [11]:
c1 = Node('child/c1')
c1

Node('child/c1')

In [12]:
root.add_child(c1)

In [13]:
root.children
print(c1.parent)
del root
print(c1.parent)

Node('parent/root')
